# Scraping The Surface of Mars

In [75]:
# Import Dependencies
import IPython
from IPython.display import display_html
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd

In [76]:
# Mars URL page to be scraped for news title and paragraph text
site_news_url = 'https://mars.nasa.gov/news/'

In [77]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(site_news_url)

In [78]:
html = browser.html
soup = bs(html, 'html.parser')

In [79]:
mars_subtags = soup.select_one('ul.item_list li.slide')

In [80]:
# Latest News Title
news_title = mars_subtags.find('div', class_='content_title').get_text()
news_title

'MOXIE Could Help Future Rockets Launch Off Mars'

In [81]:
# Paragraph Text
news_parag = mars_subtags.find('div', class_='article_teaser_body').get_text()
news_parag

"NASA's Perseverance rover carries a device to convert Martian air into oxygen that, if produced on a larger scale, could be used not just for breathing, but also for fuel."

In [82]:
browser.quit()

# Mars Featured Image

In [83]:
# Mars URL page to be scraped for image
mars_site_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

In [84]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# Navagate with Browser to Largesize Image
browser.visit(mars_site_image_url)
browser.links.find_by_partial_text('FULL IMAGE').click()
browser.links.find_by_partial_text('more info').click()
browser.links.find_by_partial_href('/largesize/').click()

# Get featured image
mars_featured_image_html = browser.html
mars_featured_image_soup = bs(mars_featured_image_html, 'html.parser')

mars_featured_image_url = mars_featured_image_soup.select_one('img')['src']
browser.quit()
mars_featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17896_hires.jpg'

In [85]:
large_image = mars_featured_image_url
print(large_image)
IPython.display.HTML('<img src=' + large_image + ' width=55%>')

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17896_hires.jpg


# Mars Facts

In [86]:
# Assign Mars Facts webpage url to variant
mars_facts_url = 'https://space-facts.com/mars/'

In [87]:
mars_facts_tables = pd.read_html(mars_facts_url)

In [88]:
mars_facts_df = mars_facts_tables[0]
mars_facts_df.columns = ['Attributes', 'Data']

In [89]:
# Display Mars Facts Dataframe
mars_facts_df.set_index('Attributes', inplace=True)
mars_facts_df

,Data
Attributes,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [90]:
# Convert the data to a HTML table string and save to file
mars_facts_df.to_html('templates/mars_fact_table.html')

In [91]:
# Convert dataframe to HTML string and display
mars_facts = mars_facts_df.to_html(header=True, index=True).replace('\n', '')
display_html(mars_facts, raw=True)

,Data
Attributes,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


# Mars Hemispheres

In [92]:
# USGS Astrogeology site
astrogeoogy_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

In [93]:
# Initiate browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

browser.visit(astrogeoogy_url)

In [94]:
# Create Empty Lists to Save image urls and Hemisphere titles
mars_hemisphere_image_urla = []
mars_hemisphere_image_urlb = []

mars_hemispheres = browser.find_by_css('a.product-item h3')
for hem in range(len(mars_hemispheres)):

    browser.find_by_css('a.product-item h3')[hem].click()
    
    sample_image = browser.links.find_by_text('Sample').first

# ---- dict a -----
    hemisphere_dicta = {
        'img_url' : sample_image['href'],
        'title' : browser.find_by_css('h2.title').text
    }
    
# ---- dict b -----
    hemisphere_dictb = {
        'Mars Hemisphere' : browser.find_by_css('h2.title').text.replace(' Hemisphere Enhanced',''),
        'URL' : sample_image['href']
    }
    
    mars_hemisphere_image_urla.append(hemisphere_dicta)
    mars_hemisphere_image_urlb.append(hemisphere_dictb)
    
    browser.back()

browser.quit()

pd.set_option('display.max_colwidth', None)

mars_hemisphere_df = pd.DataFrame.from_dict(mars_hemisphere_image_urlb)
mars_hemisphere_df.set_index('Mars Hemisphere', inplace=True)
display(mars_hemisphere_df)
print('-' * 42 + '  Mars Hemisphere Images  ' + '-' * 42)
display(mars_hemisphere_image_urla)

,URL
Mars Hemisphere,
Cerberus,https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli,https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major,https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris,https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


------------------------------------------  Mars Hemisphere Images  ------------------------------------------


[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [131]:
image_a = IPython.display.HTML('<img src=' + mars_hemisphere_df['URL'][0] + ' width=20% style="float:left; margin:0px 15px 15px">'+
                               '<img src=' + mars_hemisphere_df['URL'][1] + ' width=20% style="float:left; margin:0px 15px 15px">'+
                               '<img src=' + mars_hemisphere_df['URL'][2] + ' width=20% style="float:left; margin:0px 15px 15px">'+
                               '<img src=' + mars_hemisphere_df['URL'][3] + ' width=20% style="float:left; margin:0px 15px 15px">')
display(image_a)